## IBM Applied Data Science Capstone Project by Coursera

Our aim is to find the areas in the neighbourhood of Delhi, India which are best suited for opening a sweet shop

- Get the data of the neighbourhoods of Delhi from wikipedia
- Use the geocoder to get the coordinates of the neighbourhoods
- Use the folium to put coordinates on map
- Use foursquare API to explore the neighbourhood
- Explore the cluster using K- means
- Choose the cluster which is best for opening the sweet shop in Delhi


Install the libraries needed

In [1]:
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.0.1         | 575 KB    | ##################################### | 1

## Import the libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


Import the data by web scraping the neighbourhoods of delhi from wikipedia

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.findAll("li"):
    neighborhoodList.append(row.text)
    # create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

df=kl_df[12:197].reset_index(drop=True)
df.head()

,Neighborhood
0,Adarsh Nagar
1,Ashok Vihar
2,Azadpur
3,Bawana
4,Begum Pur


In [5]:
df.shape

(185, 1)

Get the coordinates of the neighbourhoods

In [6]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
cords=[]
for neighborhood in df["Neighborhood"].tolist() :
    cords.append(get_latlng(neighborhood))
cords

[[28.720350000000053, 77.17264000000006],
 [28.69037000000003, 77.17609000000004],
 [28.712420000000066, 77.17304000000007],
 [28.800590000000057, 77.03473000000008],
 [28.732620000000054, 77.05213000000003],
 [28.707092837280925, 77.20451789081991],
 [28.728770000000054, 77.17557000000005],
 [28.737330000000043, 77.03959000000003],
 [28.688010000000077, 77.15866000000005],
 [28.711690000000033, 77.20197000000007],
 [28.69659000000007, 77.13892000000004],
 [28.705010000000073, 77.18950000000007],
 [28.83979000000005, 77.07696000000004],
 [28.69590000000005, 77.13725000000005],
 [28.68584000000004, 77.13188000000008],
 [28.719470000000058, 77.10147000000006],
 [28.733560000000068, 77.10401000000007],
 [28.714230000000043, 77.15744000000007],
 [28.67557000000005, 77.16190000000006],
 [28.660990000000027, 77.20450000000005],
 [28.656270000000063, 77.23232000000007],
 [28.648580000000038, 77.23071000000004],
 [28.67671000000007, 77.21767000000006],
 [28.65457890544559, 77.23339989939495],


In [7]:
df_coords = pd.DataFrame(cords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(185, 3)


,Neighborhood,Latitude,Longitude
0,Adarsh Nagar,28.720350,77.172640
1,Ashok Vihar,28.690370,77.176090
2,Azadpur,28.712420,77.173040
3,Bawana,28.800590,77.034730
4,Begum Pur,28.732620,77.052130
5,Dhaka,28.707093,77.204518
6,Jahangirpuri,28.728770,77.175570
7,Karala,28.737330,77.039590
8,Keshav Puram,28.688010,77.158660
9,Kingsway Camp,28.711690,77.201970


In [8]:
df.to_csv("delhi_df.csv", index=False)

Create the map of Delhi and superimpose the neighbourhoods on it

In [9]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6273928, 77.1716954.


In [10]:
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [11]:
map_dl.save('map_dl.html')

Use foursquare API to explore the neighbourhood.

In [35]:
# @hidden cell
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [13]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

Dataframe of the explored neighbours

In [14]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(7155, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72035,77.17264,Shalimar bagh club,28.711882,77.157933,Indian Restaurant
1,Adarsh Nagar,28.72035,77.17264,Ever Bake,28.706419,77.162607,Bakery
2,Adarsh Nagar,28.72035,77.17264,Arjun Pao Bhaji,28.708468,77.184963,Indian Restaurant
3,Adarsh Nagar,28.72035,77.17264,Domino's Pizza,28.706000,77.163000,Pizza Place
4,Adarsh Nagar,28.72035,77.17264,Cafe Coffee Day,28.707934,77.161478,Coffee Shop


Groupby the data with neighbourhood

In [15]:
venues_df.groupby(["Neighborhood"])['VenueName'].count()

Neighborhood
Adarsh Nagar           12
Alaknanda              55
Anand Vihar            32
Ashok Nagar             6
Ashok Vihar            22
Azadpur                25
Babarpur                5
Badarpur                5
Bali Nagar             59
Bara Hindu Rao         66
Barakhamba Road       100
Bawana                  1
Begum Pur               2
Brij Puri               8
Chanakyapuri           78
Chandni Chowk         118
Chawri Bazaar         100
Chhattarpur            31
Chittaranjan Park      94
Civil Lines            35
Connaught Place       100
Dabri, New Delhi        5
Dariba Kalan           60
Daryaganj              39
Dayanand Colony       100
Defence Colony         93
Delhi Cantonment        9
Dhaka                  26
Dhaula Kuan            86
Dilshad Garden         10
Dwarka Sub City        33
East Vinod Nagar       10
East of Kailash       100
Fateh Nagar            48
Friends Colony         15
Gandhi Nagar            4
Gautampuri              9
Geeta Colony            5

In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
# print out the list of categories
venues_df['VenueCategory'].unique()

There are 212 uniques categories.


array(['Indian Restaurant', 'Bakery', 'Pizza Place', 'Coffee Shop',
       'Light Rail Station', 'Park', 'Bus Station', 'Bank', 'Restaurant',
       'Golf Course', 'Athletics & Sports', 'Asian Restaurant',
       'Sandwich Place', 'Snack Place', 'South Indian Restaurant',
       'Department Store', 'Fast Food Restaurant', 'Market',
       'Basketball Court', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Multiplex', 'Donut Shop',
       'Convenience Store', 'Gym / Fitness Center', 'Shopping Mall',
       'Playground', 'Tourist Information Center', 'Café',
       'Chinese Restaurant', 'Food Truck', 'Burger Joint',
       'Italian Restaurant', 'Ice Cream Shop', 'History Museum', 'Diner',
       'Arts & Crafts Store', 'BBQ Joint', 'Hotel', 'Tea Room', 'ATM',
       'Mobile Phone Shop', 'Clothing Store', 'Theme Park',
       'Food & Drink Shop', 'Dessert Shop', 'Miscellaneous Shop', 'Motel',
       'Flea Market', 'Bed & Breakfast', 'Museum', 'Mosque', 'Boutique',
       'Monumen

In [17]:
"Indian Sweet Shop" in venues_df['VenueCategory'].unique()

True

Analysis of the data

In [18]:
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(7155, 213)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Weight Loss Center,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Group the data by neighbourhood and take the mean of the frequency the occourence of venue

In [19]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(172, 213)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Weight Loss Center,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00000,0.000000,0.0

Number of area having the sweet shop

In [23]:
len(dl_grouped[dl_grouped["Indian Sweet Shop"] > 0])

7

In [24]:
dl_sweetshop = dl_grouped[["Neighborhoods","Indian Sweet Shop"]]
dl_sweetshop.head()

,Neighborhoods,Indian Sweet Shop
0,Adarsh Nagar,0.0
1,Alaknanda,0.0
2,Anand Vihar,0.0
3,Ashok Nagar,0.0
4,Ashok Vihar,0.0


Applying the K Means clustering

In [25]:
kclusters = 3

dl_clustering = dl_sweetshop.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl_merged = dl_sweetshop.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

Data gets segmented in three parts

In [27]:
dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Indian Sweet Shop,Cluster Labels
0,Adarsh Nagar,0.0,0
1,Alaknanda,0.0,0
2,Anand Vihar,0.0,0
3,Ashok Nagar,0.0,0
4,Ashok Vihar,0.0,0


In [28]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dl_merged = dl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head() # check the last columns!

(182, 5)


,Neighborhood,Indian Sweet Shop,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.0,0,28.72035,77.17264
1,Alaknanda,0.0,0,28.52567,77.25325
2,Anand Vihar,0.0,0,28.65356,77.31373
3,Ashok Nagar,0.0,0,28.69223,77.30124
4,Ashok Vihar,0.0,0,28.69037,77.17609


In [29]:
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(182, 5)


,Neighborhood,Indian Sweet Shop,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.720350,77.172640
111,Partap Nagar,0.000000,0,28.669770,77.195750
112,Paschim Vihar,0.000000,0,28.669330,77.091730
113,Patel Nagar,0.000000,0,28.647830,77.164490
114,Patparganj,0.000000,0,28.629480,77.307660
115,Pitam Pura,0.000000,0,28.695900,77.137250
116,Pragati Maidan,0.000000,0,28.612530,77.245090
117,Pratap Nagar,0.000000,0,28.669770,77.195750
118,Preet Vihar,0.000000,0,28.639030,77.295970
110,Pandav Nagar,0.000000,0,28.614580,77.275740


Map showing different cluster with different colors

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [32]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Sweet Shop,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.720350,77.172640
111,Partap Nagar,0.000000,0,28.669770,77.195750
112,Paschim Vihar,0.000000,0,28.669330,77.091730
113,Patel Nagar,0.000000,0,28.647830,77.164490
114,Patparganj,0.000000,0,28.629480,77.307660
115,Pitam Pura,0.000000,0,28.695900,77.137250
116,Pragati Maidan,0.000000,0,28.612530,77.245090
117,Pratap Nagar,0.000000,0,28.669770,77.195750
118,Preet Vihar,0.000000,0,28.639030,77.295970
110,Pandav Nagar,0.000000,0,28.614580,77.275740


In [33]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Sweet Shop,Cluster Labels,Latitude,Longitude
7,Badarpur,0.2,1,28.50738,77.30346
6,Babarpur,0.2,1,28.50738,77.30346


In [34]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Sweet Shop,Cluster Labels,Latitude,Longitude
162,Tughlaqabad,0.142857,2,28.51858,77.27589


## Observation: 
From the above results we can say that the cluser 0 has the highest number of sweet shop in the locality and cluster 1 and 2 has no or very few sweet shop in the locality hence area that are in cluster 1 or 2 are great places for opening new sweet shop in Delhi. So builders, youngsters or any businessman could florish their sweet shop business in the area of cluser 1 or 2 because of the least competition. Opening sweet shop in cluser 0 would lead to oversupply and high competition.